<a href="https://colab.research.google.com/github/Dajounin/DRNN-Chaos/blob/master/DRNN_Chaos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from keras.models import Sequential
from keras.layers import  Dense, Dropout, Masking, Embedding
from keras.layers import GRU, LSTM
from keras.layers.core import Dense, Activation, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras import metrics
from keras.callbacks import Callback

#Auxiliar ploteo

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
plt.rcParams["figure.figsize"] = (20,10)

#Definición de sistemas caóticos

In [ ]:
class ChaosAttractors():
    def __init__(self, steps, lrz_s=10, lrz_r=28, lrz_b=8/3, lrz_dt = 0.01, 
                 rab_fab_a = 0.14, rab_fab_g = 0.1, rab_fab_dt = 0.01,
                 ros_a=0.2, ros_b=0.2, ros_c=6.3, ros_dt = 0.01):
        self.lrz_s = lrz_s
        self.lrz_b = lrz_b
        self.lrz_r = lrz_r
        self.lrz_dt = lrz_dt
        self.rab_fab_a = rab_fab_a
        self.rab_fab_g = rab_fab_g
        self.rab_fab_dt = rab_fab_dt
        self.ros_a = ros_a
        self.ros_b = ros_b
        self.ros_c = ros_c
        self.ros_dt = ros_dt
        self.steps = steps
        
    """Lorenz 63 System"""    
    def lorenz63(self):
        xs = np.empty((self.steps + 1,))
        ys = np.empty((self.steps + 1,))
        zs = np.empty((self.steps + 1,))
        
        xs[0], ys[0], zs[0] = (1.0, 1.0, 1.0)
        for i in range(self.steps):
            x_dot = self.lrz_s*(ys[i] - xs[i])
            y_dot = self.lrz_r*xs[i] - ys[i] - xs[i]*zs[i]
            z_dot = xs[i]*ys[i] - self.lrz_b*zs[i]
            xs[i + 1] = xs[i] + (x_dot * self.lrz_dt)
            ys[i + 1] = ys[i] + (y_dot * self.lrz_dt)
            zs[i + 1] = zs[i] + (z_dot * self.lrz_dt)
        return xs, ys, zs
    
    """Rabinovich–Fabrikant equations"""
    def rabinovich_fabrikant(self):
        xs = np.zeros((self.steps))
       	ys = np.zeros((self.steps))
       	zs = np.zeros((self.steps))
       	xs[0] ,ys[0] ,zs[0] = (-1,0,0.5)
       	
       	for i in range(1,self.steps):
       		x = xs[i-1]
       		y = ys[i-1]
       		z = zs[i-1]
       		dx = y*(z - 1 + x*x) + self.rab_fab_g*x
       		dy = x*(3*z + 1 - x*x) + self.rab_fab_g *y
       		dz = -2*z*(self.rab_fab_a  + x*y)
       		xs[i] = x+self.rab_fab_dt*dx
       		ys[i] = y+self.rab_fab_dt*dy
       		zs[i] = z+self.rab_fab_dt*dz
        return xs, ys, zs
    
    """Rossler Hyperchaotic System"""
    def rossler(self):
        xs = np.empty([self.steps + 1])
        ys = np.empty([self.steps + 1])
        zs = np.empty([self.steps + 1])
        xs[0], ys[0], zs[0] = (1.0, 1.0, 1.0)
        
        for i in range(self.steps):
            x_dot = -ys[i] - zs[i]
            y_dot = xs[i] + self.ros_a*ys[i]
            z_dot = self.ros_b + xs[i]*zs[i] - self.ros_c*zs[i]
            xs[i+1] = xs[i] + (x_dot * self.ros_dt)
            ys[i+1] = ys[i] + (y_dot * self.ros_dt)
            zs[i+1] = zs[i] + (z_dot * self.ros_dt)
        return xs, ys, zs

#Auxiliar de tiempo

In [ ]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

#Definición de redes neuronales

In [ ]:
class NNIdentifier():
    def __init__(self, num_neurons, num_layers,optimizer, dataset, training_epochs, batch_size, attractor_name, chaos_x_series, chaos_y_series, chaos_z_series):
        self.num_neurons = num_neurons
        self.num_layers = num_layers
        self.optimizer = optimizer
        self.dataset = dataset
        self.training_epochs = training_epochs
        self.batch_size = batch_size
        self.trainX = []
        self.trainY = []
        self.testX = []
        self.testY = []
        self.attractor_name = attractor_name
        self.look_back = 1
        self.chaos_x_series = chaos_x_series
        self.chaos_y_series = chaos_y_series
        self.chaos_z_series = chaos_z_series
        self.gru_time_callback = TimeHistory()
        self.lstm_time_callback = TimeHistory()
        self.mlp_time_callback = TimeHistory()

        
    def predict_attractor(self):
        self.normalize_datase()
        relevant_gain_points, relevant_loss_points = self.train_eval_models()
        return relevant_gain_points, relevant_loss_points
    
    def create_dataset(self, dataset, look_back=1):
    	dataX, dataY = [], []
    	for i in range(len(dataset)-look_back-1):
    		a = dataset[i:(i+look_back)]
    		dataX.append(a)
    		dataY.append(dataset[i + look_back])
    	return np.array(dataX), np.array(dataY)
 
    def normalize_datase(self):
        # Normalizar Uk 
        scaler = MinMaxScaler(feature_range=(0, 1))
        dataset = scaler.fit_transform(self.dataset)
        
        # split into train and test sets
        train_size = int(len(dataset) * 0.6)
        test_size = len(dataset) - train_size
        train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
        
        # reshape into X=t and Y=t+1
        look_back = 1
        self.trainX, self.trainY = self.create_dataset(train, look_back)
        self.testX, self.testY = self.create_dataset(test, look_back)
        
        # reshape input to be [samples, time steps, features]
        self.trainX = np.reshape(self.trainX, (self.trainX.shape[0], 1, self.trainX.shape[2]))
        self.testX = np.reshape(self.testX, (self.testX.shape[0], 1, self.testX.shape[2]))
        
    def gru_model(self):
        """GRU RNN"""
        gru_model = Sequential()
        if(self.num_layers>1):
            print('Capa de entrada')
            gru_model.add(GRU(self.num_neurons, input_shape=(1,3), return_sequences = True))
            for x in range(self.num_layers):
                print('Capa intermedia '+str(x))
                gru_model.add(GRU(self.num_neurons, return_sequences = True))
                if(x == self.num_layers-1):
                    gru_model.add(GRU(self.num_neurons, return_sequences = False))
        else:
            print('Capa de entrada')
            gru_model.add(GRU(self.num_neurons, input_shape=(1,3), return_sequences = False))
        print('Capa final')
        gru_model.add(Dense(3))
        gru_model.compile(optimizer=self.optimizer, loss='mean_squared_error', metrics =['mse','acc'])
        seq_gru_model = gru_model.fit(self.trainX, self.trainY, epochs=self.training_epochs, batch_size=self.batch_size, callbacks=self.gru_time_callback)
        return gru_model, seq_gru_model
        
    def lstm_model(self):
        """LSTM RNN"""
        lstm_model = Sequential()
        if(self.num_layers>1):
            print('Capa de entrada')
            lstm_model.add(LSTM(self.num_neurons, input_shape=(1,3), return_sequences = True))
            for x in range(self.num_layers):
                print('Capa intermedia '+str(x))
                lstm_model.add(LSTM(self.num_neurons, return_sequences = True))
                if(x == self.num_layers-1):
                    lstm_model.add(LSTM(self.num_neurons, return_sequences = False))
        else:
            print('Capa de entrada')
            lstm_model.add(LSTM(self.num_neurons, input_shape=(1,3), return_sequences = False))
        print('Capa final')
        lstm_model.add(Dense(3))
        lstm_model.compile(optimizer=self.optimizer, loss='mean_squared_error', metrics =['mse','acc'])
        seq_lstm_model = lstm_model.fit(self.trainX, self.trainY, epochs=self.training_epochs, batch_size=self.batch_size, callbacks=self.lstm_time_callback)
        return lstm_model, seq_lstm_model
        
    def mlp_model(self):
        """MLP NN"""
        mlp_model = Sequential()
        print('Capa de entrada')
        mlp_model.add(Dense(self.num_neurons, input_shape=(1,3)))
        if(self.num_layers>1):
            for x in range(self.num_layers): 
                mlp_model.add(Dense(self.num_neurons))
        mlp_model.add(Flatten())
        mlp_model.add(Dense(3))
        mlp_model.compile(optimizer=self.optimizer, loss='mean_squared_error', metrics =['mse','acc'])
        seq_mlp_model = mlp_model.fit(self.trainX, self.trainY, epochs=self.training_epochs, batch_size=self.batch_size, callbacks=self.mlp_time_callback)
        return mlp_model, seq_mlp_model
        
        
    def predict_eval_model(self, model, label_nn):
        scaler = MinMaxScaler(feature_range=(0, 1))
        dataset = scaler.fit_transform(self.dataset)
        # make predictions
        trainPredict = model.predict(self.trainX)
        testPredict = model.predict(self.testX)
        # invert predictions
        trainPredict = scaler.inverse_transform(trainPredict)
        self.trainY = scaler.inverse_transform(self.trainY)
        testPredict = scaler.inverse_transform(testPredict)
        self.testY = scaler.inverse_transform(self.testY)
        # shift train predictions for plotting
        trainPredictPlot = np.empty_like(dataset)
        trainPredictPlot[:, :] = np.nan
        trainPredictPlot[self.look_back:len(trainPredict)+self.look_back, :] = trainPredict
        # shift test predictions for plotting
        testPredictPlot = np.empty_like(dataset)
        testPredictPlot[:, :] = np.nan
        testPredictPlot[len(trainPredict)+(self.look_back*2)+1:len(dataset)-1, :] = testPredict
        # get values to graph
        val_xtrain = []
        val_ytrain = []
        val_ztrain = []
        for x in range(len(trainPredictPlot)):
          val_xtrain.append(trainPredictPlot[x][0])
          val_ytrain.append(trainPredictPlot[x][1])
          val_ztrain.append(trainPredictPlot[x][2])
        
        val_xtest = []
        val_ytest = []
        val_ztest = []
        for x in range(len(testPredictPlot)):
          val_xtest.append(testPredictPlot[x][0])
          val_ytest.append(testPredictPlot[x][1])
          val_ztest.append(testPredictPlot[x][2])
        #Graph
        fig = plt.figure()
        ax = fig.gca(projection='3d')
        
        ax.plot(self.chaos_x_series, self.chaos_y_series, self.chaos_z_series, lw=0.8, label=self.attractor_name)
        ax.plot(val_xtrain, val_ytrain, val_ztrain, lw=0.5,label='Train')
        ax.plot(val_xtest, val_ytest, val_ztest, lw=0.5, label='Test')
        legend = plt.legend(loc='upper left', shadow=True, fontsize='xx-large')
        fig_size = plt.rcParams["figure.figsize"]
        ax.set_xlabel("X Axis", fontsize=20)
        ax.set_ylabel("Y Axis", fontsize=20)
        ax.set_zlabel("Z Axis", fontsize=20)
        ax.set_title(self.attractor_name + ' - '+label_nn)
        plt.rcParams["figure.figsize"] = (20,10)
        plt.show()
      
    def graph_eval_model(self, gru_train_loss, lstm_train_loss, mlp_train_loss, gru_train_acc, lstm_train_acc, mlp_train_acc):
        """Loss evaluation and graph"""
        xc = range(self.training_epochs)
        plt.figure()
        plt.plot(xc, gru_train_loss, label='MSE - GRU')
        plt.plot(xc, lstm_train_loss, label='MSE - LSTM')
        plt.plot(xc, mlp_train_loss, label='MSE - MLP')
        plt.xlabel('Epochs')
        plt.ylabel('Error %')
        plt.yscale('log')
        plt.title('MSE for the '+ self.attractor_name)
        legend = plt.legend(loc='upper right', shadow=True, fontsize='x-large')
        plt.grid(True)
        plt.show()
        """Accuracy evaluation and graph"""
        plt.figure()
        plt.plot(xc, gru_train_acc, label ='Accuracy - GRU')
        plt.plot(xc, lstm_train_acc, label ='Accuracy - LSTM')
        plt.plot(xc, mlp_train_acc, label ='Accuracy - MLP')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy %')
        plt.title('Accuracy for the '+ self.attractor_name+' with '+str(self.num_layers)+' layers - '+str(self.num_neurons)+' neurons')
        legend = plt.legend(loc='lower right', shadow=True, fontsize='x-large')
        plt.grid(True)
        plt.show()
        
    def eval_model(self, model, seqModel):
        loss_and_metrics = model.evaluate(self.testX, self.testY, batch_size=self.batch_size)
        train_loss = seqModel.history['mse']
        train_acc  = seqModel.history['acc']
        return train_loss, train_acc
    
    def train_eval_models(self):
        """Train NN Models"""
        gru_model, seq_gru_model = self.gru_model()
        lstm_model, seq_lstm_model = self.lstm_model()
        mlp_model, seq_mlp_model = self.mlp_model()
        
        """Eval NN Models"""
        gru_train_loss, gru_train_acc = self.eval_model(gru_model, seq_gru_model)
        lstm_train_loss, lstm_train_acc = self.eval_model(lstm_model, seq_lstm_model)
        mlp_train_loss, mlp_train_acc = self.eval_model(mlp_model, seq_mlp_model)
        
        relevant_gain_points = [gru_train_acc[-1], lstm_train_acc[-1], mlp_train_acc[-1]]
        relevant_loss_points = [gru_train_loss[-1], lstm_train_loss[-1], mlp_train_loss[-1]]
        
        return relevant_gain_points, relevant_loss_points

#Función principal

In [ ]:
def create_dataset(x,y,z):
    dataset = []
    for i in range(len(x)):
        dataset.append([x[i], y[i], z[i]])
    return dataset

def eval_attractors(data_x, data_y, data_z, num_layers, num_neurons, name_chaotic_system):
    dataset = create_dataset(data_x, data_y, data_z)
    data_analysis = {}
    data_time_analysis= {}
    num_neuron = 0
    for y in range(len(num_layers)):
        print('num capas '+str(num_layers[y]))
        gain_points = []
        loss_points = []
        for x in range(len(num_neurons)):
            print('num neuronas '+str(num_neurons[x]))
            num_neuron = num_neurons[x]
            nn_identifier = NNIdentifier(num_neurons[x], num_layers[y], 'adam' ,dataset,10,32,name_chaotic_system, data_x, data_y, data_z)
            relevant_gain_points, relevant_loss_points = nn_identifier.predict_attractor()
            gain_points.append(relevant_gain_points)
            loss_points.append(relevant_loss_points)
            info_test = 'Num_neurons ['+str(num_neurons[x])+'] with ['+str(num_layers[y])+'] layers'

            data_time_analysis.update({info_test:{'GRU':{'ENLAPSED_TIME':sum(nn_identifier.gru_time_callback.times)},'LSTM':{'ENLAPSED_TIME':sum(nn_identifier.lstm_time_callback.times)},'MLP':{'ENLAPSED_TIME':sum(nn_identifier.mlp_time_callback.times)}}})
    
        gru_loss = []
        gru_gain = []
        lstm_loss = []
        lstm_gain = []
        mlp_loss = []
        mlp_gain = []
        
        for gain_point in gain_points:
            gru_gain.append(gain_point[0])
            lstm_gain.append(gain_point[1])
            mlp_gain.append(gain_point[2])
            
        for loss_point in loss_points:
            gru_loss.append(loss_point[0])
            lstm_loss.append(loss_point[1])
            mlp_loss.append(loss_point[2])

        info_test = 'Layer_'+str(num_layers[y])
        data_analysis.update({info_test: {'GRU':{'Loss':gru_loss,'Gain':gru_gain},'LSTM':{'Loss':lstm_loss,'Gain':lstm_gain},'MLP':{'Loss':mlp_loss,'Gain':mlp_gain}}})
        

    return data_analysis, data_time_analysis

        

num_layers = [5, 8, 10]      
num_neurons = [32, 64, 128, 256]
num_epochs = 10
attractors_series  = ChaosAttractors(10000)
lorenz_x, lorenz_y, lorenz_z = attractors_series.lorenz63()
attractors_series  = ChaosAttractors(50000)
rab_x, rab_y, rab_z = attractors_series.rabinovich_fabrikant()
ros_x, ros_y, ros_z = attractors_series.rossler()
data_analysis_gru = eval_attractors(lorenz_x, lorenz_y, lorenz_z,num_layers, num_neurons, 'Lorenz 63 System')
data_analysis_lstm = eval_attractors(rab_x, rab_y, rab_z,num_layers, num_neurons, 'Rabinovich-Fabrikant System')
data_analysis_mlp = eval_attractors(ros_x, ros_y, ros_z,num_layers, num_neurons, 'Rossler System')

num capas 5
num neuronas 32
Capa de entrada
Capa intermedia 0
Capa intermedia 1
Capa intermedia 2
Capa intermedia 3
Capa intermedia 4
Capa final
Epoch 1/10
188/188 [==============================] - 1s 8ms/step - loss: 0.0421 - mse: 0.0421 - acc: 0.5183
Epoch 2/10
188/188 [==============================] - 1s 8ms/step - loss: 0.0083 - mse: 0.0083 - acc: 0.6909
Epoch 3/10
188/188 [==============================] - 2s 8ms/step - loss: 0.0016 - mse: 0.0016 - acc: 0.7337
Epoch 4/10
188/188 [==============================] - 1s 8ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.8011
Epoch 5/10
188/188 [==============================] - 2s 8ms/step - loss: 9.6994e-04 - mse: 9.6994e-04 - acc: 0.8710
Epoch 6/10
188/188 [==============================] - 2s 9ms/step - loss: 9.1308e-04 - mse: 9.1308e-04 - acc: 0.8566
Epoch 7/10
188/188 [==============================] - 2s 9ms/step - loss: 8.0531e-04 - mse: 8.0531e-04 - acc: 0.8846
Epoch 8/10
188/188 [==============================] - 1s 8ms/step - 

In [ ]:
print(data_analysis_gru)
print(data_analysis_lstm)
print(data_analysis_mlp)

({'Layer_5': {'GRU': {'Loss': [0.0005211507668718696, 4.4722015445586294e-05, 4.609927782439627e-05, 0.00018767578876577318], 'Gain': [0.8919639587402344, 0.9643214344978333, 0.962487518787384, 0.9556518793106079]}, 'LSTM': {'Loss': [0.007252024486660957, 0.006400474812835455, 0.0005054667708463967, 0.0004117892822250724], 'Gain': [0.6885628700256348, 0.6993998289108276, 0.8919639587402344, 0.8912971019744873]}, 'MLP': {'Loss': [0.00013197121734265238, 0.0001540214434498921, 0.00017675827257335186, 0.0001814627757994458], 'Gain': [0.9436478614807129, 0.9424808025360107, 0.9398132562637329, 0.9326441884040833]}}, 'Layer_8': {'GRU': {'Loss': [0.0007006563246250153, 0.0004032352298963815, 0.00045023090206086636, 0.0002959711418952793], 'Gain': [0.8837946057319641, 0.8942980766296387, 0.8902967572212219, 0.9058019518852234]}, 'LSTM': {'Loss': [0.007982727140188217, 0.006370058748871088, 0.006145489867776632, 0.0005392785533331335], 'Gain': [0.6927309036254883, 0.7095698714256287, 0.7410803